# Quantum Integer Programming (QuIP) 47-779. Fall 2020, CMU

## Introduction to Mathematical Programming
### Modeling
The solution of optimization problems requires the development of a mathematical model. Here we will model an example given in the lecture and see how an integer program can be solved practically. This example will use as modeling language Pyomo (http://www.pyomo.org/), an open-source Python package, which provides a flexible access to different solvers and a general modeling framework for linear and nonlinear integer programs.
The examples solved here will make use of open-source solvers GLPK (https://www.gnu.org/software/glpk/) for linear and mixed-integer linear programming and IPOPT (https://coin-or.github.io/Ipopt/) for nonlinear programming.

#### Problem statement
Suppose there is a company that produces two different product, A and B, which can be sold at different values, $5.5 and $2.1 per unit respectively.
The company only counts with a single machine with electricity usage of at most 17kW/day; and producing each A and B consumes 8kW/day and 2kW/day, respectively.
Besides, the company can only produce at most 2 more units of A than B per day.

This is a valid model, but it would be easier to solve if we had a mathematical representation.
Assuming the units produced of A are $x_1$ and of B are $x_2$ we have
$$
\max_{x_1, x_2} 5.5x_1 + 2.1x_2 \\
s.t. x_2 \leq x_1 + 2 \\
8x_1 + 2x_2 \leq 17 \\
x_1, x_2 \geq 0
$$


In [2]:
# Import the Pyomo library, which can be installed via pip, conda or from Github https://github.com/Pyomo/pyomo
import pyomo.environ as pyo

In [4]:
# Define the model
model = pyo.ConcreteModel(name='Simple example LP, 47-779 QuIP')
#Define the variables
model.x = pyo.Var([1,2], domain=pyo.NonNegativeReals)
# Define the objective function
model.obj = pyo.Objective(expr = 5.5*model.x[1] + 2.1*model.x[2], sense=pyo.maximize)
# Define the constraints
model.Constraint1 = pyo.Constraint(expr = model.x[2] <= model.x[1] + 2)
model.Constraint2 = pyo.Constraint(expr = 8*model.x[1] + 2*model.x[2] <= 17)
# Print the model
model.pprint()

1 Set Declarations
    x_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    2 : {1, 2}

1 Var Declarations
    x : Size=2, Index=x_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          1 :     0 :  None :  None : False :  True : NonNegativeReals
          2 :     0 :  None :  None : False :  True : NonNegativeReals

1 Objective Declarations
    obj : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : maximize : 5.5*x[1] + 2.1*x[2]

2 Constraint Declarations
    Constraint1 : Size=1, Index=None, Active=True
        Key  : Lower : Body              : Upper : Active
        None :  -Inf : x[2] - (x[1] + 2) :   0.0 :   True
    Constraint2 : Size=1, Index=None, Active=True
        Key  : Lower : Body            : Upper : Active
        None :  -Inf : 8*x[1] + 2*x[2] :  17.0 :   True

5 Declarations: x_index x obj Constraint1 Constraint2


In [6]:
# Define the solver
opt = pyo.SolverFactory('glpk')
# Here we could use another solver, e.g. gurobi or cplex
# opt = pyo.SolverFactory('gurobi')

In [8]:
# Here we solve the optimization problem, the option tee=True prints the solver output
result_obj = opt.solve(model, tee=False)

In [10]:
# Display solution of the problem
model.display()

Model Simple example LP, 47-779 QuIP

  Variables:
    x : Size=2, Index=x_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          1 :     0 :   1.3 :  None : False : False : NonNegativeReals
          2 :     0 :   3.3 :  None : False : False : NonNegativeReals

  Objectives:
    obj : Size=1, Index=None, Active=True
        Key  : Active : Value
        None :   True : 14.08

  Constraints:
    Constraint1 : Size=1
        Key  : Lower : Body : Upper
        None :  None :  0.0 :   0.0
    Constraint2 : Size=1
        Key  : Lower : Body : Upper
        None :  None : 17.0 :  17.0


We observe that the optimal solution of this problem is $x_1 = 1.3$, $x_2 = 3.3$, leading to a profit of $14.08$.

Now let's consider that only integer units of each product can be produced, namely
$$
\max_{x_1, x_2} 5.5x_1 + 2.1x_2 \\
s.t. x_2 \leq x_1 + 2 \\
8x_1 + 2x_2 \leq 17 \\
x_1, x_2 \geq 0 \\
x_1, x_2 \in \mathbb{Z}
$$

In [16]:
# Define the model
model_int = pyo.ConcreteModel(name='Simple example IP, 47-779 QuIP')
#Define the variables
model_int.x = pyo.Var([1,2], domain=pyo.Integers)
# Define the objective function
model_int.obj = pyo.Objective(expr = 5.5*model_int.x[1] + 2.1*model_int.x[2], sense=pyo.maximize)
# Define the constraints
model_int.Constraint1 = pyo.Constraint(expr = model_int.x[2] <= model_int.x[1] + 2)
model_int.Constraint2 = pyo.Constraint(expr = 8*model_int.x[1] + 2*model_int.x[2] <= 17)
# Print the model
model_int.pprint()

1 Set Declarations
    x_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    2 : {1, 2}

1 Var Declarations
    x : Size=2, Index=x_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          1 :  None :  None :  None : False :  True : Integers
          2 :  None :  None :  None : False :  True : Integers

1 Objective Declarations
    obj : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : maximize : 5.5*x[1] + 2.1*x[2]

2 Constraint Declarations
    Constraint1 : Size=1, Index=None, Active=True
        Key  : Lower : Body              : Upper : Active
        None :  -Inf : x[2] - (x[1] + 2) :   0.0 :   True
    Constraint2 : Size=1, Index=None, Active=True
        Key  : Lower : Body            : Upper : Active
        None :  -Inf : 8*x[1] + 2*x[2] :  17.0 :   True

5 Declarations: x_index x obj Constraint1 Constraint2


In [19]:
# Here we solve the optimization problem, the option tee=True prints the solver output
result_obj_int = opt.solve(model_int, tee=False)

In [20]:
model_int.display()

Model Simple example IP, 47-779 QuIP

  Variables:
    x : Size=2, Index=x_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          1 :  None :   1.0 :  None : False : False : Integers
          2 :  None :   3.0 :  None : False : False : Integers

  Objectives:
    obj : Size=1, Index=None, Active=True
        Key  : Active : Value
        None :   True :  11.8

  Constraints:
    Constraint1 : Size=1
        Key  : Lower : Body : Upper
        None :  None :  0.0 :   0.0
    Constraint2 : Size=1
        Key  : Lower : Body : Upper
        None :  None : 14.0 :  17.0
